In [ ]:
#FoMAC_Autoformer.ipynb
from MarkovAutoformer import MarkovAutoformer, MarkovAutoformerConfig
from markov_autoformer_pipeline import prepare_data, MarkovAutoformerTrainer, calculate_metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from joblib import load, dump
from torch.optim.lr_scheduler import LambdaLR
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# LOSS FUNCTIONS
class TrendLoss(nn.Module):
    def __init__(self, alpha=0.8):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.alpha = alpha

    def forward(self, pred, true):
        loss_val = self.l1(pred, true)
        trend_penalty = torch.mean(
            (torch.sign(pred[:, 1:, :] - pred[:, :-1, :]) != 
             torch.sign(true[:, 1:, :] - true[:, :-1, :])).float()
        )
        return loss_val + self.alpha * trend_penalty


class HybridHuberTrendLoss(nn.Module):
    def __init__(self, alpha=0.8, beta=0.5, delta=1.0):
        super().__init__()
        self.huber = nn.HuberLoss(delta=delta)
        self.trend = TrendLoss(alpha)
        self.beta = beta

    def forward(self, pred, true):
        huber_loss = self.huber(pred, true)
        trend_loss = self.trend(pred, true)
        return (1 - self.beta) * huber_loss + self.beta * trend_loss

In [ ]:
class EnhancedMarkovAutoformerTrainer(MarkovAutoformerTrainer):
    
    def train_epoch(self, train_loader, optimizer, criterion):
        self.model.train()
        total_main_loss = 0.0
        total_markov_loss = 0.0
        total_combined_loss = 0.0
        
        for batch_idx, batch in enumerate(train_loader):
            seq_x, seq_x_mark, seq_y, seq_y_mark, seq_y_target, seq_state_x, seq_state_y = batch
            
            seq_x = seq_x.to(self.device)
            seq_x_mark = seq_x_mark.to(self.device)
            seq_y = seq_y.to(self.device)
            seq_y_mark = seq_y_mark.to(self.device)
            seq_y_target = seq_y_target.to(self.device)
            seq_state_x = seq_state_x.to(self.device)
            seq_state_y = seq_state_y.to(self.device)
            
            optimizer.zero_grad()
            
            outputs = self.model(seq_x, seq_x_mark, seq_y, seq_y_mark,
                               seq_state_x=seq_state_x, seq_state_y=seq_state_y)
            
            if outputs.shape[-1] > 1:
                outputs = outputs[..., 0:1]
            
            pred = outputs if not isinstance(outputs, (tuple, list)) else outputs[0]
            
            pred_len = min(pred.shape[1], seq_y_target.shape[1])
            pred = pred[:, -pred_len:, :]
            target = seq_y_target[:, -pred_len:, :]
            main_loss = criterion(pred, target)
            
            markov_loss = self._compute_state_supervision_loss(seq_state_x)
            
            weight = getattr(self.model, 'markov_supervised_weight', 0.3)
            loss = main_loss + weight * markov_loss
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_main_loss += main_loss.item()
            total_markov_loss += markov_loss.item()
            total_combined_loss += loss.item()
        
        return {
            'combined': total_combined_loss / max(1, len(train_loader)),
            'main': total_main_loss / max(1, len(train_loader)),
            'markov': total_markov_loss / max(1, len(train_loader))
        }
    
    def validate(self, val_loader, criterion):
        self.model.eval()
        total_main_loss = 0.0
        total_markov_loss = 0.0
        total_combined_loss = 0.0
        
        with torch.no_grad():
            for batch in val_loader:
                seq_x, seq_x_mark, seq_y, seq_y_mark, seq_y_target, seq_state_x, seq_state_y = batch
                
                seq_x = seq_x.to(self.device)
                seq_x_mark = seq_x_mark.to(self.device)
                seq_y = seq_y.to(self.device)
                seq_y_mark = seq_y_mark.to(self.device)
                seq_y_target = seq_y_target.to(self.device)
                seq_state_x = seq_state_x.to(self.device)
                seq_state_y = seq_state_y.to(self.device)
                
                outputs = self.model(seq_x, seq_x_mark, seq_y, seq_y_mark,
                                   seq_state_x=seq_state_x, seq_state_y=seq_state_y)
                
                if outputs.shape[-1] > 1:
                    outputs = outputs[..., 0:1]
                
                pred = outputs if not isinstance(outputs, (tuple, list)) else outputs[0]
                pred_len = min(pred.shape[1], seq_y_target.shape[1])
                pred = pred[:, -pred_len:, :]
                target = seq_y_target[:, -pred_len:, :]
                main_loss = criterion(pred, target)
                
                markov_loss = self._compute_state_supervision_loss(seq_state_x)
                weight = getattr(self.model, 'markov_supervised_weight', 0.3)
                loss = main_loss + weight * markov_loss
                
                total_main_loss += main_loss.item()
                total_markov_loss += markov_loss.item()
                total_combined_loss += loss.item()
        
        return {
            'combined': total_combined_loss / max(1, len(val_loader)),
            'main': total_main_loss / max(1, len(val_loader)),
            'markov': total_markov_loss / max(1, len(val_loader))
        }


In [ ]:
# DATA PREPARATION

print("=" * 80)
print("PREPARING DATA")
print("=" * 80)

train_loader, val_loader, test_loader, scalers_dict, feature_cols = prepare_data(
    features_path='feature_engineered_data.csv',
    seq_len=120,
    label_len=14,
    pred_len=14,
    batch_size=16,
    stride=1,
    train_ratio=0.8,
    val_ratio=0.1,
    location='MANILA CITY',
)

# Filter to most relevant features
selected_features = [
    # Epidemic
    'cases_minmax', 'cases_minmax_lag_1w', 'cases_minmax_lag_2w',
    'cases_minmax_lag_3w', 'cases_minmax_lag_4w', 'cases_minmax_roll_mean_4w',
    
    # Rainfall
    'RAINFALL_minmax', 'RAINFALL_minmax_lag_1w', 'RAINFALL_minmax_lag_2w',
    'RAINFALL_minmax_lag_3w', 'RAINFALL_minmax_roll_mean_4w',
    'RAIN_L6', 'RAIN_L8', 'RAIN_L12',
    
    # Temperature
    'TMAX_minmax', 'TMIN_minmax', 'TMAX_minmax_lag_1w', 'TMIN_minmax_lag_1w',
    'TMAX_minmax_roll_mean_4w', 'TMIN_minmax_roll_mean_4w', 'TMAX_x_RH',
    'TMAX_L6', 'TMAX_L8', 'TMAX_L12',
    
    # Wind
    'WIND_SPEED_minmax', 'WIND_DIR_X', 'WIND_DIR_Y',
    
    # Humidity
    'RH_minmax', 'RH_minmax_lag_1w', 'RH_minmax_roll_mean_4w',
    'RH_L6', 'RH_L8', 'RH_L12',
    
    # Time encodings
    'month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos'
]

feature_cols = [f for f in feature_cols if f in selected_features]

print(f"\n✓ Using {len(feature_cols)} selected features")
print(f"✓ Train batches: {len(train_loader)}")
print(f"✓ Val batches: {len(val_loader)}")
print(f"✓ Test batches: {len(test_loader)}")

In [ ]:
# MODEL CONFIGURATION

print("\n" + "=" * 80)
print("CONFIGURING MODEL")
print("=" * 80)

config = MarkovAutoformerConfig()

# Sequence parameters
config.seq_len = 120
config.label_len = 14
config.pred_len = 14

# Input/output dimensions
config.enc_in = len(feature_cols)
config.dec_in = len(feature_cols)
config.c_out = 1

# === MODEL SIZE ===
config.d_model = 256
config.n_heads = 8
config.e_layers = 3
config.d_layers = 2
config.d_ff = 1024

# Regularization
config.dropout = 0.5
config.moving_avg = 13

# === MARKOV PARAMETERS ===
config.n_states = 16
config.markov_weight = 0.4
config.markov_tau = 0.8

# === STATE SUPERVISION ===
config.markov_supervised_weight = 0.4

config.embed = 'timeF'
config.freq = 'w'
config.activation = 'gelu'
config.output_attention = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MarkovAutoformer(config)

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"\n✓ Model initialized on {device}")
print(f"✓ Total trainable parameters: {total_params:,}")
print(f"✓ Expected memory: ~{total_params * 4 / 1e6:.1f} MB")

In [ ]:
# TRAINING SETUP

print("\n" + "=" * 80)
print("TRAINING SETUP")
print("=" * 80)

trainer = EnhancedMarkovAutoformerTrainer(model, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

def lr_lambda(epoch):
    return min(1.0, (epoch + 1) / 20)
warmup_scheduler = LambdaLR(optimizer, lr_lambda)

main_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=30, T_mult=2
)

criterion = HybridHuberTrendLoss(alpha=0.8, beta=0.6, delta=1.0)

epochs = 200
best_val_loss = float('inf')
patience = 10
patience_counter = 0

train_losses = {'combined': [], 'main': [], 'markov': []}
val_losses = {'combined': [], 'main': [], 'markov': []}

In [ ]:
# TRAINING LOOP

print("\n" + "=" * 80)
print("STARTING TRAINING (WITH STATE SUPERVISION)")
print("=" * 80)

patience = 15       
counter = 0         


best_val_loss = float('inf')

for epoch in range(epochs):
    # Train
    train_loss_dict = trainer.train_epoch(train_loader, optimizer, criterion)
    
    # Validate
    model.eval()
    with torch.no_grad():
        val_loss_dict = trainer.validate(val_loader, criterion)
    model.train()
    
    # Scheduler
    if epoch < 10:
        warmup_scheduler.step()
    else:
        main_scheduler.step(epoch)
    
    # Track losses
    for key in train_losses.keys():
        train_losses[key].append(train_loss_dict[key])
        val_losses[key].append(val_loss_dict[key])
    
    # Compute 3-epoch moving average for validation
    if len(val_losses['combined']) >= 3:
        recent_avg = sum(val_losses['combined'][-3:]) / 3
    else:
        recent_avg = val_loss_dict['combined']
    
    # --- CHECK FOR IMPROVEMENT ---
    if recent_avg < best_val_loss - 1e-5:
        best_val_loss = recent_avg
        torch.save(model.state_dict(), 'best_markov_autoformer.pth')
        improvement_flag = "✓ IMPROVED"
        counter = 0  # Reset counter on success
    else:
        counter += 1 # Increment counter on failure
        improvement_flag = f"  (Wait {counter}/{patience})"
    
    # Logging
    if (epoch + 1) % 5 == 0 or epoch == 0 or counter >= patience:
        print(f"Epoch {epoch+1:3d}/{epochs} | "
              f"Train: {train_loss_dict['combined']:.5f} "
              f"(Main: {train_loss_dict['main']:.5f}, State: {train_loss_dict['markov']:.5f}) | "
              f"Val: {val_loss_dict['combined']:.5f} "
              f"(Main: {val_loss_dict['main']:.5f}, State: {val_loss_dict['markov']:.5f}) "
              f"{improvement_flag}")

    # --- BREAK IF PATIENCE EXCEEDED ---
    if counter >= patience:
        print("\n" + "!" * 40)
        print(f"EARLY STOPPING TRIGGERED AT EPOCH {epoch+1}")
        print("!" * 40)
        break

print("\n" + "=" * 80)
print(f"✓ Training complete! Best val loss: {best_val_loss:.6f}")
print("=" * 80)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

# Plot training & validation loss
ax.plot(train_losses['combined'], label='Training Loss', linewidth=2)
ax.plot(val_losses['combined'], label='Validation Loss', linewidth=2)

# Unified title
ax.set_title('Training and Validation Loss over Epochs')

# Unified axis labels
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')

ax.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))

# Legend
ax.legend()

# Visible grid
ax.grid(True, alpha=0.4)

plt.tight_layout()
plt.show()


In [ ]:
# EVALUATION

print("\n" + "=" * 80)
print("EVALUATING ON TEST SET")
print("=" * 80)

model.load_state_dict(torch.load('best_markov_autoformer.pth'))
predictions, targets = trainer.predict(test_loader)

print(f"✓ Predictions shape: {predictions.shape}")
print(f"✓ Targets shape: {targets.shape}")

# Inverse transform to original scale
predictions = np.array(predictions)
targets = np.array(targets)

df = pd.read_csv('feature_engineered_data.csv')
cases_raw = df['cases'].values.reshape(-1, 1)

scaler_cases = MinMaxScaler()
scaler_cases.fit(cases_raw)

pred_target = predictions[..., 0:1]
true_target = targets[..., 0:1]

pred_target = np.clip(pred_target, 0.0, 1.0)

pred_real = scaler_cases.inverse_transform(pred_target.reshape(-1, 1)).reshape(pred_target.shape)
true_real = scaler_cases.inverse_transform(true_target.reshape(-1, 1)).reshape(true_target.shape)


# Compute metrics
def compute_metrics(true_vals, pred_vals):
    true_vals = true_vals.reshape(-1)
    pred_vals = pred_vals.reshape(-1)
    
    mse = mean_squared_error(true_vals, pred_vals)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true_vals, pred_vals)
    r2 = r2_score(true_vals, pred_vals)
    smape = 100 * np.mean(2 * np.abs(pred_vals - true_vals) / 
                          (np.abs(true_vals) + np.abs(pred_vals) + 1e-8))
    corr = np.corrcoef(true_vals, pred_vals)[0, 1]
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'SMAPE': smape, 'Correlation': corr}

metrics_real = compute_metrics(true_real, pred_real)

print("\nMETRICS ON ORIGINAL DENGUE CASE COUNTS:")
for k, v in metrics_real.items():
    print(f"  {k:12s}: {v:.4f}")

print("\nMETRICS ON SCALED (0–1) CASE VALUES:")

pred_scaled = pred_target.reshape(-1)
true_scaled = true_target.reshape(-1)

scaled_mse = mean_squared_error(true_scaled, pred_scaled)
scaled_rmse = np.sqrt(scaled_mse)
scaled_mae = mean_absolute_error(true_scaled, pred_scaled)

print(f"  MSE (scaled):   {scaled_mse:.4f}")
print(f"  RMSE (scaled):  {scaled_rmse:.4f}")
print(f"  MAE (scaled):   {scaled_mae:.4f}")
print(f"  Range of pred_scaled: {pred_scaled.min():.4f} to {pred_scaled.max():.4f}")
print(f"  Range of true_scaled: {true_scaled.min():.4f} to {true_scaled.max():.4f}")

In [ ]:
# VISUALIZATION

true_real = np.array(true_real).flatten()
pred_real = np.array(pred_real).flatten()

# Actual vs Predicted
plt.figure(figsize=(14, 5))
plt.plot(true_real[:200], label='Actual', linewidth=2.5, color='black', alpha=0.7)
plt.plot(pred_real[:200], label='Predicted', linewidth=2, linestyle='--', color='dodgerblue')
plt.title('Dengue Cases Prediction (With State Supervision)', fontsize=14)
plt.xlabel('Time Steps')
plt.ylabel('Dengue Cases')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("✓ Training and evaluation complete!")
print("✓ States were used for supervision throughout training")
print("=" * 80)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Parameters
seq_len = 120
pred_len = 52
stride = 1

# Fake timeline (just integers for visualization)
timeline = np.arange(1, 200)

# Windows for visualization (first 3 windows)
win1 = timeline[0:seq_len]
win2 = timeline[stride:stride+seq_len]
win3 = timeline[2*stride:2*stride+seq_len]

fig, ax = plt.subplots(figsize=(14, 4))

# Plot horizontal bars to represent windows
ax.plot(win1, np.ones_like(win1)*3, label="Window 1: Weeks 1–120", linewidth=8)
ax.plot(win2, np.ones_like(win2)*2, label="Window 2: Weeks 2–121", linewidth=8)
ax.plot(win3, np.ones_like(win3)*1, label="Window 3: Weeks 3–122", linewidth=8)

# Formatting
ax.set_title("Sliding Window Visualization (Stride = 1)", fontsize=14)
ax.set_xlabel("Week Index")
ax.set_yticks([1,2,3])
ax.set_yticklabels(["Window 3","Window 2","Window 1"])
ax.grid(True, linestyle='--', alpha=0.4)
ax.legend()

plt.tight_layout()
plt.show()
